In [ ]:
#will be using FINBERT model for sentiment analysis

In [1]:
!pip install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/8d/10/8cef2288810a3210659eb3a20711e8387cc35a881a7762ae387806e2d651/transformers-4.53.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.30.0 from https://files.pythonhosted.org/packages/44/f4/5f3f22e762ad1965f01122b42dae5bf0e009286e2dba601ce1d0dba72424/huggingface_hub-0.33.2-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/7a/d1/598de10b17fdafc452d11f7dada11c3be4e379a8671393e4e3da3c4070df/regex-2024.11.6-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.9 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.22,>=0.21 from https://files.pythonhosted.org/packages/6c/e6/33f41f2cc7861faeba8988e7a77601407bf1d9d28fc79c5903f8f77df587/t

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
model.eval()


/Users/vladpavlovich/PycharmProjects/LSM-FinBert-Trader/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [3]:
def get_sentiment_score(texts):
    scores = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            score = probs[0][2] - probs[0][0]  # positive - negative
            scores.append(score.item())
    return sum(scores) / len(scores) if scores else 0.0


In [7]:
import requests

def fetch_apple_news(date, api_key):
    """
    Fetches news headlines for Apple on a given date.
    Returns a list of headline strings.
    """
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": "Apple OR AAPL",
        "from": date,
        "to": date,
        "language": "en",
        "sortBy": "relevancy",
        "pageSize": 30,
        "apiKey": "c69d24631c9249a6ad530e984ba3889e"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if "articles" in data:
        return [article["title"] for article in data["articles"]]
    else:
        print("Error:", data.get("message", "Unknown error"))
        return []


In [8]:
api_key = "c69d24631c9249a6ad530e984ba3889e"  # 

from datetime import datetime

# Today's date
date = datetime.today().strftime('%Y-%m-%d')

headlines = fetch_apple_news(date, api_key)
print("Headlines:", headlines)

sentiment = get_sentiment_score(headlines)
print(f"{date} sentiment score:", sentiment)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Headlines: ['Pilou Asbæk on Playing the Mule, ‘Foundation’ Season 3’s Terrifying Villain', 'The Ploopy Knob is an open-source control dial for your PC', "The MacRumors Show: Apple's Plan to Launch Low-Cost MacBook With iPhone Chip", 'iPhone 17 Pro Coming Soon With These 14 New Features', 'When will F1 movie be on Apple TV? Here’s the streaming release date', '4th of July Apple sale: Save up to $800 on hardware, monitors & more', "Siri's future, the original iPhone's past, and Apple Music's birthday on the AppleInsider Podcast", 'All of the best 4th of July Apple deals', 'Apple just released a weirdly interesting coding language model', "Here's why the Oura Ring 4 is a postpartum mum's unexpected best friend", 'The iPhone 17 Pro will wirelessly charge your Apple Watch and AirPods, according to news leaks', 'A rare look inside the durability lab where Apple tortures its products', 'These were the most-watched Apple TV+ movies and shows in June', "Tim Cook isn't going to get fired, and St